# Introduction
I am interested in exploring how COVID-19 and the lockdowns in San Franciso affected the amount of crime in the city. I will be comparing crime reports in 2020 with those in 2018 and 2019. Specifically I am interested in looking at:
1. The amount of crime reported in the city and how it changed throughout 2020
2. The change in the category of crime being committed 
3. How the trend of when crimes usually take place changes with the lockdowns

Dataset from DataSF: https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783

# Import Packages

In [150]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Explore and Clean the Data

In [151]:
df = pd.read_csv('SF_Crime.csv')
print(f"The dataset has {df.shape[1]} columns and {df.shape[0]} rows")
print("*****"*10)
df.info()

The dataset has 36 columns and 422717 rows
**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422717 entries, 0 to 422716
Data columns (total 36 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   Incident Datetime                                     422717 non-null  object 
 1   Incident Date                                         422717 non-null  object 
 2   Incident Time                                         422717 non-null  object 
 3   Incident Year                                         422717 non-null  int64  
 4   Incident Day of Week                                  422717 non-null  object 
 5   Report Datetime                                       422717 non-null  object 
 6   Row ID                                                422717 non-null  int64  
 7   Incident ID                       

### Description of Columns
|Column Name|Description|
|:--- |:--- |
|Incident Datetime|The date and time when the incident occurred|
|Incident Date|The date the incident occurred|
|Incident Time|The time the incident occurred|
|Incident Year|The year the incident occurred, provided as a convenience for filtering|
|Incident Day of Week|The day of week the incident occurred|
|Report Datetime|Distinct from Incident Datetime, Report Datetime is when the report was filed.|
|Row ID|A unique identifier for each row of data in the dataset|
|Incident ID|This is the system generated identifier for incident reports. Incident IDs and Incident Numbers both uniquely identify reports, but Incident Numbers are used when referencing cases and report documents.|
|Incident Number|The number issued on the report, sometimes interchangeably referred to as the Case Number. This number is used to reference cases and report documents.|
|CAD Number|The Computer Aided Dispatch (CAD) is the system used by the Department of Emergency Management (DEM) to dispatch officers and other public safety personnel. CAD Numbers are assigned by the DEM system and linked to relevant incident reports (Incident Number). Not all Incidents will have a CAD Number. Those filed online via Coplogic (refer to “Filed Online” field) and others not filed through the DEM system will not have CAD Numbers.|
|Report Type Code|A system code for report types, these have corresponding descriptions within the dataset.|
|Report Type Description|The description of the report type, can be one of: Initial; Initial Supplement; Vehicle Initial; Vehicle Supplement; Coplogic Initial; Coplogic Supplement|
|Filed Online|Non- emergency police reports can be filed online by members of the public using SFPD’s self-service reporting system called Coplogic Values in this field will be “TRUE” if Coplogic was used to file the report.|
|Incident Code|Incident Codes are the system codes to describe a type of incident. A single incident report can have one or more incident types associated. In those cases you will see multiple rows representing a unique combination of the Incident ID and Incident Code.|
|Incident Category|A category mapped on to the Incident Code used in statistics and reporting. Mappings provided by the Crime Analysis Unit of the Police Department.|
|Incident Subcategory|A subcategory mapped to the Incident Code that is used for statistics and reporting. Mappings are provided by the Crime Analysis Unit of the Police Department.|
|Incident Description|The description of the incident that corresponds with the Incident Code. These are generally self-explanatory.|
|Resolution|The resolution of the incident at the time of the report. Can be one of: • Cite or Arrest Adult • Cite or Arrest Juvenile* • Exceptional Adult • Exceptional Juvenile* • Open or Active •|
|Intersection|The 2 or more street names that intersect closest to the original incident separated by a backward slash (\). Note, the possible intersections will only include those that satisfy the privacy controls.|
|CNN|The unique identifier of the intersection for reference back to other related basemap datasets.|
|Police District|The Police District where the incident occurred. |
|Analysis Neighborhood|This field is used to identify the neighborhood where each incident occurs. Neighborhoods and boundaries are defined by the Department of Public Health and the Mayor's Office of Housing and Community Development.|
|Supervisor District|There are 11 members elected to the Board of Supervisors in San Francisco, each representing a geographic district. The Board of Supervisors is the legislative body for San Francisco. The districts are numbered 1 through 11.|
|Latitude|The latitude coordinate in WGS84, spatial reference is EPSG:4326|
|Longitude|The longitude coordinate in WGS84, spatial reference is EPSG:4326|
|point|The point geometry used for mapping features in the open data portal platform. Latitude and Longitude are provided separately as well as a convenience.|


In [152]:
df.head(5)

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,HSOC Zones as of 2018-06-05,OWED Public Spaces,Central Market/Tenderloin Boundary Polygon - Updated,Parks Alliance CPSI (27+TL sites),ESNCAG - Boundary File,"Areas of Vulnerability, 2016"
0,2018/01/01 09:26:00 AM,2018/01/01,09:26,2018,Monday,2018/01/01 09:27:00 AM,61893007041,618930,171052174,173641140.0,...,88.0,2.0,9.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0
1,2018/01/01 02:30:00 AM,2018/01/01,02:30,2018,Monday,2018/01/01 08:21:00 AM,61893105041,618931,180000768,180010668.0,...,90.0,9.0,1.0,7.0,NaN,NaN,NaN,NaN,NaN,2.0
2,2018/01/01 10:00:00 AM,2018/01/01,10:00,2018,Monday,2018/01/01 10:20:00 AM,61893275000,618932,180000605,180010893.0,...,20.0,4.0,10.0,36.0,NaN,NaN,1.0,NaN,NaN,2.0
3,2018/01/01 10:03:00 AM,2018/01/01,10:03,2018,Monday,2018/01/01 10:04:00 AM,61893565015,618935,180000887,180011579.0,...,NaN,9.0,1.0,28.0,NaN,NaN,NaN,NaN,NaN,1.0
4,2018/01/01 09:01:00 AM,2018/01/01,09:01,2018,Monday,2018/01/01 09:39:00 AM,61893607041,618936,171052958,180011403.0,...,106.0,6.0,3.0,6.0,NaN,NaN,NaN,NaN,NaN,2.0


I want to check which columns contain null values and how many null values there are in the dataset

In [153]:
print(f"There are {df.isna().values.sum()} null values")
print("*****"*10)
print(df.isnull().sum())

There are 2636476 null values
**************************************************
Incident Datetime                                            0
Incident Date                                                0
Incident Time                                                0
Incident Year                                                0
Incident Day of Week                                         0
Report Datetime                                              0
Row ID                                                       0
Incident ID                                                  0
Incident Number                                              0
CAD Number                                               93453
Report Type Code                                             0
Report Type Description                                      0
Filed Online                                            336865
Incident Code                                                0
Incident Category                    

In the columns I'm interested in, the only columns that contains null values are Incident Category and Incident Subcategory. There are only 331 null values though so I should be able to complete my analysis.

Next I will check cells with categorial variables to make sure we don't have any misspellings or inputs that shouldn't be there.

In [154]:
print(f"Incident Year: {sorted(df['Incident Year'].unique())}")
print("")
print(f"Incident Day of Week: {sorted(df['Incident Day of Week'].unique())}")
print("")
print(f"Report Type Code: {sorted(df['Report Type Code'].unique())}")
print("")
print(f"Report Type Description: {sorted(df['Report Type Description'].unique())}")
print("")
print(f"Filed Online: {sorted(df['Filed Online'].unique())}")
print("")
df['Incident Category'] = df['Incident Category'].astype(str)
print(f"Incident Category: {sorted(df['Incident Category'].unique())}")
print("")
df['Incident Subcategory'] = df['Incident Subcategory'].astype(str)
print(f"Incident Subcategory: {sorted(df['Incident Subcategory'].unique())}")
print("")
print(f"Resolution: {sorted(df['Resolution'].unique())}")
print("")
print(f"Police District: {sorted(df['Police District'].unique())}")
print("")
print(f"Supervisor District: {sorted(df['Supervisor District'].unique())}")

Incident Year: [2018, 2019, 2020, 2021]

Incident Day of Week: ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']

Report Type Code: ['II', 'IS', 'VI', 'VS']

Report Type Description: ['Coplogic Initial', 'Coplogic Supplement', 'Initial', 'Initial Supplement', 'Vehicle Initial', 'Vehicle Supplement']

Filed Online: [nan, True]

Incident Category: ['Arson', 'Assault', 'Burglary', 'Case Closure', 'Civil Sidewalks', 'Courtesy Report', 'Disorderly Conduct', 'Drug Offense', 'Drug Violation', 'Embezzlement', 'Family Offense', 'Fire Report', 'Forgery And Counterfeiting', 'Fraud', 'Gambling', 'Homicide', 'Human Trafficking (A), Commercial Sex Acts', 'Human Trafficking (B), Involuntary Servitude', 'Human Trafficking, Commercial Sex Acts', 'Larceny Theft', 'Liquor Laws', 'Lost Property', 'Malicious Mischief', 'Miscellaneous Investigation', 'Missing Person', 'Motor Vehicle Theft', 'Motor Vehicle Theft?', 'Non-Criminal', 'Offences Against The Family And Children', 'Othe

The 'Incident Category' seems to have a few mistakes:
- 'Human Trafficking' has 3 different types of inputs
- 'Motor Vehicle Theft' input has 2 different types of inputs, and one has a question mark. This seems to be an error
- 'Weapons Offense' is also spelled 'Offence' at times

I will want to look closer into the 'Incident Category' column to see how many times each value appears

In [155]:
def count_unique_values(df,column):   
    column_values = {}
    for i in df[column]:
        if i in column_values.keys():
            column_values[i] = column_values.get(i, 0) + 1
        else:
            column_values[i] = 0
    df_column_values = pd.DataFrame.from_dict(column_values, orient='index').reset_index().rename(columns = {'index':'category',0:'num'})
    return df_column_values.sort_values('category')

count_unique_values(df,'Incident Category')

,category,num
30,Arson,1126
4,Assault,25044
1,Burglary,22412
33,Case Closure,1883
10,Civil Sidewalks,860
36,Courtesy Report,1230
21,Disorderly Conduct,7205
17,Drug Offense,10413
27,Drug Violation,135
34,Embezzlement,586


There are very few 'Moter Vehicle Theft?' and 'Weapons Offence' entries so I think they may have been an input error. I will combine them with their respective categories. I will also combine all of the 'Human Trafficking' categories into one since there are so few of them.

I am going to group all of the 'Other' categories together. There are so many different categories, I may want to only look at the top 20-25 categories by crime numbers and group the rest of them with the 'Other' category.

Next I'll check to see that we don't have duplicate rows in the database

In [156]:
def find_duplicate_rows(df):
    df_duplicates = df[df.duplicated()]
    if df_duplicates.shape[0] > 0:
        return(df_duplicates)
    else:
        return("There are no duplicate rows")

print(find_duplicate_rows(df))

There are no duplicate rows


There are no duplicate rows but I also want to make sure that there are no Incident IDs that are being used more than once. According to the column description, if an incident report has more than one incident type associated, there will be additional rows with unique combinations of the Incident ID and Incident Code. So now I'll want to look to see if there are any rows that have the same combination of Incident ID and Incident Code

In [157]:
def find_duplicate_column_combos(df,*columns):
    column_list = []
    for column in columns:
        column_list.append(column)
    df_duplicates = df[df.duplicated(columns)]
    if df_duplicates.shape[0] > 0:
        return(f"There are {df_duplicates.shape[0]} duplicate column combinations")
    else:
        return("There are no duplicate column combinations")

print(find_duplicate_column_combos(df,'Incident ID','Incident Code'))

There are no duplicate column combinations


# Manipulating the Data Using SQL
Aggregating and filtering the data for visualizing in Tableau
``` mysql
SELECT incidentyear, 
       incidentmonth, 
       incidentday, 
       incidentdayofweek, 
       CASE 
         WHEN rank > 20 THEN 'Other' 
         ELSE incidentcategory 
       END AS incidentcategory, 
       resolution, 
       policedistrict, 
       SUM(num_daily_reports) AS num_daily_reports 
FROM   (SELECT *, 
               DENSE_RANK() OVER (ORDER BY total_reports DESC) AS rank 
        FROM   (SELECT *, 
                       SUM(num_daily_reports) OVER 
                          (PARTITION BY incidentcategory) AS total_reports 
                FROM   (SELECT incidentyear, 
                               CAST(SUBSTR(incidentdate, 6, 2) AS INTEGER) 
                                   AS IncidentMonth, 
                               CAST(SUBSTR(incidentdate, 9, 2) AS INTEGER) 
                                   AS IncidentDay, 
                               incidentdayofweek, 
                               CASE 
                                 WHEN incidentcategory = 'Weapons Offence' 
                                     THEN 'Weapons Offense' 
                                 WHEN incidentcategory = 'Motor Vehicle Theft?' 
                                     THEN 'Motor Vehicle Theft' 
                                 WHEN incidentcategory LIKE '%Human Trafficking%' 
                                     THEN 'Human Trafficking, Commercial Sex Acts' 
                                 WHEN incidentcategory LIKE 'Other%' 
                                     THEN 'Other'
                                 ELSE incidentcategory 
                               END AS incidentcategory, 
                               resolution, 
                               policedistrict, 
                               COUNT(*) AS 
                               num_daily_reports 
                        FROM   sf_crime 
                        WHERE  incidentyear != 2021 
                        GROUP  BY 1,2,3,4,5,6,7))) 
GROUP  BY 1,2,3,4,5,6,7  
```

# Tableau Visualizations and Analysis

## Amount of Crime
First let's look at how the amount of crime in 2020 compares to 2018 and 2019

In [158]:
%%html
<div class='tableauPlaceholder' id='viz1611336840637' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;TotalCrimeDash&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SFCrime_16106615948520&#47;TotalCrimeDash' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;TotalCrimeDash&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1611336840637');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

The cumulative amount of reported crimes in 2020 was around 23% less than the average of 2018 and 2019. We can see that 2020 starts to diverge in March, when COVID-19 was declared a pandemic and SF enacted a shelter-in-place order. Let's take a closer look at March 2020 to see exactly when crime started to decrease.

In [159]:
%%html
<div class='tableauPlaceholder' id='viz1611336880940' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;March2020Dash&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SFCrime_16106615948520&#47;March2020Dash' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;March2020Dash&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1611336880940');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

So pretty much right when COVID-19 was declared a national emergency in the US, SF crime rates started to trend downwards. 

I would like to see how the amount of crime differs by month compared to the year's average.

In [160]:
%%html
<div class='tableauPlaceholder' id='viz1611336916795' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;DifferencefromAverageDash&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SFCrime_16106615948520&#47;DifferencefromAverageDash' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;DifferencefromAverageDash&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1611336916795');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

We generally see less crime during the winter and more crime in the warmer months from July to October (SF's 'Indian Summer'). Given the pandemic, 2020's monthly trend is different, but I want to remove January through April to see if the rest of the year resembles 2018 and 2019. 

When filtering out Jan-April, although we do see an uptick in crimes starting in the summer, the number of crimes continues to be higher as we go into the winter. I think the unpredictability of COVID-19 and government restrictions really affected the number of crimes in the beginning. Then as the pandemic went on, the public may have started to experience pandemic fatigue and relaxed adherence to regulations, therefore providing more opportunities for crimes to occur. 

Next I will want to look at what types of crimes occurred more frequently in 2020 and how that differs from previous years due to COVID-19.

## Categories of Crime
First I want to see how the total amount of each category of crime changed in 2020 compared to previous years.

In [161]:
%%html
<div class='tableauPlaceholder' id='viz1611346368173' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;DifferenceinTotalCrimebyYear&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SFCrime_16106615948520&#47;DifferenceinTotalCrimebyYear' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;DifferenceinTotalCrimebyYear&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1611346368173');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

There are a few observations that I would like to call out:
- Although Larceny Theft continued to be the #1 crime, it still dropped a huge amount from 2019 to 2020.
- Malicious Michief had very minimal change compared to other major categories.
- Even though we saw a decrease in overall crime in 2020, three categories still saw increases in crime: Burglary, Motor Vehicle Theft, and Weapons Offense.
- Lost Property dropped the most out of any category from 2019 to 2020.

Below is just a general graph of the amount of crime reported per day for the entire year of 2020. You can click through the categories filter to see how each one changed before and after COVID-19 was declared a national emergency in March.

In [162]:
%%html
<div class='tableauPlaceholder' id='viz1611337012805' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;DailyCategoryCrimeDash&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SFCrime_16106615948520&#47;DailyCategoryCrimeDash' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;DailyCategoryCrimeDash&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1611337012805');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Now I want to look at how the rankings of the top 15 crime categories have changed over the course of the years by comparing the number of crimes committed in January to the number committed in December.

In [163]:
%%html
<div class='tableauPlaceholder' id='viz1611337167602' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;CategoryRankbyYearDash&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SFCrime_16106615948520&#47;CategoryRankbyYearDash' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;CategoryRankbyYearDash&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1611337167602');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

For 2018 and 2019, there are some changes in the ranks from January to December. However the changes are not as dramatic as the ranking changes we see in 2020. About half or fewer of the categories switched ranks in 2018 and 2019, and only by 1 or 2 spots. In 2020, the rankings of all of the top 15 categories, except for Larceny Theft, have changed over the course of the year. Some categories even jumped anywhere from 3 to 6 spots.

Let's look at 2020 month by month to see how the start of the pandemic may have affected which crime categories were more prevalent.

In [164]:
%%html
<div class='tableauPlaceholder' id='viz1611117566565' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;CrimeCategoryRankDash&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SFCrime_16106615948520&#47;CrimeCategoryRankDash' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;CrimeCategoryRankDash&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1611117566565');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='977px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

We see a lot of volatility in March and April. Most notably:
- Burglary and Motor Vehicle Theft have taken the #2 and #3 ranks, respectively
- Lost Property dropped from #7 all the way down to #11 and continued to rank low for the rest of the year
- Fraud's ranking went up starting in March, and was #6 before drastically dropping to #10 in the last 2 months of 2020
- Robbery initially went down in March, and continued to rank #10 until the very end of the year where it jumped to #6
- Offences Against the Family and Children moved up 4 ranks March - April. Although its rank dropped a bit, the final 2020 ranking is still higher than it was in the beginning of the year

I am also interested in comparing the percentage makeup of crime by category throughout the years.

To gain a better understanding of why COVID-19 and restrictions may have caused changes in these crime categories for 2020, I want to look closer at what the sub-categories of these crimes are.

In [165]:
%%html
<div class='tableauPlaceholder' id='viz1611282165756' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;CrimeSubcategoriesDash&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SFCrime_16106615948520&#47;CrimeSubcategoriesDash' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;CrimeSubcategoriesDash&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1611282165756');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1000px';vizElement.style.height='727px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1000px';vizElement.style.height='727px';} else { vizElement.style.width='100%';vizElement.style.height='877px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

##### Changes by crime category and personal theories:
- **Burglary:** Starting in April we see an increase in crime in all subcategories. The large increase in Commercial Burglary in May coincides with when individuals were taking advantage of the BLM protests and breaking into stores. It's strange that residential burglaries would go up during a period when a majority of people are stuck in their homes. I wonder if more burglaries were noticed and reported because more people were staying at home. In researching some criminology, it seems there are two different theories when it comes to crime: “opportunity theory” and “strain theory”. Opportunity theory speculates that crime will go down because restrictions prevent opportunities for crime. On the other hand, strain theory speculates that crime will go up because socioeconomic strains will push people to commit more crimes out of necessity. I wonder if the increase we're seeing with burglaries in San Francisco is a result of strain opportunity. San Francisco has a huge divide in wealth due to the tech bubble. Even without a recession and job losses, many in the city are unable to afford living there. I imagine a lockdown and record job losses just exacerbates the problem.
<br><br>
- **Fraud:** One of the largest categories, False Personation, stays at similar levels to previous years until Nov-Dec. However, starting in March, we see a drop in what is usually one of the largest categories of fraud: Access Card, incl. Credit, Phone, ATM, Fraudulent Use of. We also see a drop in Fraudulent Use of Automated Teller Card. It seems that since people are out less, and thus using their credit and debit cards less frequently, opportunities for these types of fraud have gone down.
<br><br>
- **Larceny Theft:** Although we see a decrease across most subcategories, the most noticable changes are to theft from vehicle and other theft. It seems that since there are fewer people out and about, there are much fewer opportunities for theives. This could correlate with the increase in home burglaries. Because theives are unable to pickpocket out on the streets, they have to resort to breaking into homes.
<br><br>
- **Lost Property:** So this may not actually be categorized under crimes. However the decrease in lost property being reported could be similar to the reason for Larceny Theft's decrease: everyone is confined to their homes, therefore there are fewer opportunities to lose anything.
<br><br>
- **Malicious Mischief:** It doesn't seem like there is much of a change in the amount of Malicious Mischief compared to previous years. I guess if someone is going to go out and break the law, they aren't going to follow restrictions anyways. Social restrictions don't reduce the number of things to vandalize.
<br><br>
- **Motor Vehicle Theft:** This category saw a surge going into the summer, and again at the end of the year. With a majority of people stuck at home with nowhere to go, many cars are probably left unattended for weeks at a time. This gives thieves a higher chance of stealing cars while going unnoticed. 
<br><br>
- **Robbery:** We see an overall decrease in Robbery, but Commercial Robbery seems to stay at about the same levels as previous years. While there were fewer opportunities for pedestrian robberies during the pandemic, many stores that were deemed essential were open and thus still targets for robberies. In November and December we actually see Commercial Robberies double. I wonder if this could be related to the criminology "strain theory" as more and more people are desperate to support themselves. Or maybe restrictions were lifted for some more stores, thus more opportunities given for robberies.



## Crime by Hour
Finally I want to look at what times crimes were more likely to occur and how this differs in 2020 from past years.

In [166]:
%%html
<div class='tableauPlaceholder' id='viz1611344718527' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;CrimebyHourDash&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SFCrime_16106615948520&#47;CrimebyHourDash' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;SF&#47;SFCrime_16106615948520&#47;CrimebyHourDash&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1611344718527');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1000px';vizElement.style.height='627px';} else { vizElement.style.width='100%';vizElement.style.height='777px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Crime generally follows the same flow in 2020 as in previous years. However there's a noticable increase in crime in the early morning hours from 2am to 6am. We also see a decrease in crime from 5pm to 8pm (17-20). When looking at the categories individually, I see 2 noticable changes from 2018/2019 to 2020 that would cause this change in hourly crime:
1. Larceny Theft decreased by ~38.5% from 2019 to 2020. Since this category made up a majority of the crime in the afternoon and evening, the large drop in crime significantly impacted the distribution of crime throughout the day.
2. Burglary was one of the few categories to increase in 2020. It seems that because most people were staying at home, residential buglaries increased during the early morning hours, when most people would be sleeping.

## Summary and Take-Aways

With the onset of the COVID-19 pandemic combined with changes to social and economic norms, we see two popular criminology theories at play in San Francisco: Opportunity Theory and Strain Theory. Opportunity theory speculates that crime will go down because restrictions prevent opportunities for crime. On the other hand, strain theory proposes that crime will go up because socioeconomic strains will push people to commit more crimes out of necessity. 
<br><br>
**Opportunity Theory**<br>
We do see an overall decrease in crime in 2020 due to lack of opportunties. This is especially evident in crime categories such as larceny theft, assault, street robberies, disorderly conduct, and traffic violations. As more people are confined to their homes, there are much fewer targets for criminals. As a result, the amount of crime in many of categories dropped by 20% or more from 2019. Larceny theft, the largest category by number of reports, dropped by 38.5%. This massive drop caused overall crime in SF to be around 23% less than the amount of crime in 2018 and 2019. One category actually did not see much of a change from previous years to 2020: malicious mischief. The main subcategory for malicious mischief is vandalism. For this category, I believe it remained unchanged in 2020 because these types of crimes can still be done even if a majority of people are stuck at home. Thus, the opportunity was still there. The types of people who would commit vandalism may also be the types who wouldn't follow government lockdown guidelines.
<br><br>
**Strain Theory**<br>
There are just a few crime categories that actually increased due to pandemic restrictions, namely, burglary, motor vehicle theft, and commercial robbery. There is already enormous wealth inequality in San Francisco. With record job losses and very little government assistance, some who are at the bottom of the socioeconomic ladder may find themselves even more desperate to commit crimes.
- Initially, I found it strange that number of residential buglaries would have increased during a time when most people are in their homes. Wouldn't there be a higher chance of getting caught now? However, after digging in a little deeper, it seems that many of these buglaries are occurring in the very early morning hours from midnight to 6am, when most people would be sleeping. The number of burglaries from midnight to 6am increased by 79% from 2019 while burglaries during the day increased by 35%. This change in burglary patterns slightly changed the overall distribution of crime throughout the day. Now a slightly larger portion of crime occurs from 2am to 6am due to the increase in home burglaries (in addition to the decrease in larceny theft that usually occurred in the afternoon). Criminals have to resort to robbing homes in the middle of the night because it is now their only option.
- Although there were probably fewer cars out on the street, car jacking was much higher in 2020 than in previous years. There is a theory that because everyone is wearing masks, car jackers are less worried about being recognized when they approach a car. I also speculate that because cars are being left unattended for longer periods of time, car jackers are able to break into these parked cars without fear of getting caught in the act.
- Commercial robbery was actually lower in the early half of the pandemic, as any store that wasn't deemed essential was forced to stay closed. However, near the end of the year, we see commercial robbery increase as some restrictions on shops were lifted and additional government assistance about to end.
    
   
Because of pandemic-related restrictions, we see a lot of volatility in which categories of crime are more prevalent in 2020, especially in March and April when lockdowns were the most strict. In 2018 and 2019, larceny theft, assault, and malicious mischief consistently made up the top 3 crimes committed in San Francisco. However by the end of 2020, burglary and motor vehicle theft eventually took over as 2nd and 3rd, leaving larceny theft still as #1.